In [ ]:
dff = df[df['stroke'] == 1]  
# Đọc dữ liệu từ tập tin CSV
import pandas as pd
# Đường dẫn đến tập tin CSV
#file_path = 'C:\\Users\\LG\\Desktop\\BD_DQ\\data_dotquy.csv'
file_path = 'C:\\Users\\ACER\\Desktop\\BD_DQ\\data_dotquy.csv'
# Đọc dữ liệu từ CSV vào DataFrame
df = pd.read_csv(file_path)
# Hiển thị DataFrame trong VS Code
print(df)

LÀM SẠCH và CHUẨN HÓA DL
- 1 giới tính: loại other
- 2 tuổi: Loại số thực ( vd: 0.1 , 1.56) rồi dùng kỹ thuật Age Grouping
- 3 Đã kết hôn: chuyển về 0 và một
- 4 kiểu làm việc(work_type)
    + kỹ thuật Hashing Encoding
    + kỹ thuật One-Hot Encoding
- 5 vùng sống: chuyển về 0 và một
- 6 BMI  
    + Loại bỏ các hàng hoặc cột có N/A rồi dùng kỹ thuật Binary Encoding
    + dùng kỹ thuật Simple Imputation rồi dùng kỹ thuật tiếp Min-Max Scaling
- 7 smoking_status
  + Loại bỏ các hàng hoặc cột có Unknown rồi dùng kỹ thuật tiếp Min-Max Scaling
  + kỹ thuật Frequency Encoding
- 8 avg_glucose_level (mg/dL)
    DÙNG KỸ THUẬT: Min-Max Scaling
  ******ĐIỀU KIỆN LÀ******
    Người bình thường:
     < 140
    Người Bị tiểu đường:
      >=140

In [ ]:
#Hien
#Kiem tra trung lap
duplicated_rows = df[df.duplicated(subset='id')]
print("So luong du lieu trung lap la:" , len(duplicated_rows))

#df = df.dropna()
#df = df.applymap(lambda x: x.replace("\n"," ") if isinstance(x, str) else x )

In [ ]:
#hien
#Kiểm tra dữ liệu có bị thiếu
df.isnull().sum()

In [ ]:
#Hien
# dùng knn điền dữ liệu thiếu vào BMI
# Tạo đối tượng KNN Imputer với số lượng hàng xóm là 3
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)

# Áp dụng imputer cho DataFrame, chỉ chọn các cột số
df[['age', 'bmi']] = imputer.fit_transform(df[['age', 'bmi']])

# Hiển thị DataFrame sau khi điền dữ liệu
print(df)

In [ ]:
# Kiểm tra lại đã dủ dữ liệu chưa
df['bmi'].isnull().sum()
df['bmi']

In [ ]:
#Hien
# Xóa cột mã định danh
df  = df.drop(columns=['id'])
df

In [ ]:
#Chau_Muc1
#Loại bỏ giới tính mang giá trị Other
mask = df['gender'].isin(['Male', 'Female'])
df = df[mask]
print(df)

In [ ]:
#Chau_Muc1
# gender: female 1 male 0
df.loc[:, 'gender'] = df['gender'].replace({'Female': 1, 'Male': 0})
print(df)

In [ ]:
#Chau_Muc5(Phần này phân công cho GIANG)
# Residence_type: urban 1 rural 0(can note)
#df.loc[:, 'Residence_type'] = df['Residence_type'].replace({'Urban': 1, 'Rural': 0})


In [ ]:
#Chau_Muc4(Phần này lấy code của bạn GIANG)
#chuẩn hóa dữ liệu field work_type theo kỹ thuật label encoding
#from sklearn.preprocessing import LabelEncoder
#le=LabelEncoder()
#df.loc[:, 'work_type'] = le.fit_transform(df['work_type'])
#print(df)

In [ ]:
#Chi_Muc2
# Kiểm tra xem tuổi có phải là số nguyên hay không
df['Is_Whole_Number'] = df['age'].apply(lambda x: x.is_integer())

# Lọc các hàng mà tuổi không phải là số thực (chỉ giữ lại số nguyên)
df = df[df['Is_Whole_Number']]

# Xóa cột tạm thời 'Is_Whole_Number'
df = df.drop(columns=['Is_Whole_Number'])

df

In [ ]:
#Chi_Muc2
# Chuẩn hóa bằng Min-Max Scaling
# Calculate min and max values
X_min = min(df['age'])
X_max = max(df['age'])

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in df['age']]

# Gán lại dữ liệu vào cột ['avg_glucose_level']
df['age'] = scaled_data
df

In [ ]:
#Chi(điều chỉnh tuổi về min max)
# Định nghĩa các khoảng tuổi và các nhãn cho các nhóm tuổi
#bins = [0, 18, 35, 60, 100]
#labels = ['0', '1', '2', '3']

# Chia cột "Age" thành các nhóm tuổi
#df['Age Group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

#df

In [ ]:
#Chi_Muc3
#Chuyen trạng thái đã kết hôn về dạng 1 và chưa kết hôn về dạng 0
df['ever_married'] = df['ever_married'].map({'Yes': 1, 'No': 0})
df

In [ ]:
#Giang_Muc4(Gán thẳng giá trị vào work_type)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['work_type'] = label_encoder.fit_transform(df['work_type'])

# sử dụng chuẩn hóa bằng Min-Max Scaling chuan hoa df['work_type'] dang (0 1 2 3 4)
# Calculate min and max values
X_min = min(df['work_type'])
X_max = max(df['work_type'])

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in df['work_type']]
scaled_data
# Tạo ra trường dữ liệu mới work_type_2
df['work_type'] =scaled_data
df

In [ ]:
#Giang_Muc5
# Chuyen trạng thái thành thị về dạng 1 và nông thôn về dạng 0
df['Residence_type']=df['Residence_type'].replace({'Urban':1,'Rural':0})
mask = df['gender'] != 'Other'
df_filtered = df[mask]
print(df_filtered) 

In [ ]:
#Hien_Mục6
# chuẩn hóa bằng Min-Max Scaling
glucose_values = df['avg_glucose_level']

# Calculate min and max values
X_min = min(df['avg_glucose_level'])
X_max = max(df['avg_glucose_level'])

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in df['avg_glucose_level']]

# Gán lại dữ liệu vào cột ['avg_glucose_level']
df['avg_glucose_level'] = scaled_data
df


In [ ]:
#Hoang_Muc7
# Dữ liệu ban đầu
data = df['bmi']

# Min-Max Scaling
data_min = data.min()
data_max = data.max()

scaled_data = (data - data_min) / (data_max - data_min)

# Hiển thị dữ liệu đã chuẩn hóa
print(scaled_data)

In [ ]:
#Hoang_Muc7
df['bmi']=scaled_data
df

In [ ]:
#Hoang_Muc7
# Dữ liệu ban đầu
data = df['bmi']

# Chuyển đổi dữ liệu thành DataFrame
dff = pd.DataFrame(data, columns=['bmi'])
dff

In [ ]:
dff = df[df['stroke'] == 1]  
dff

In [ ]:
dfstr = df[df['heart_disease'] == 1]
dfstr

In [ ]:
dfstrss = dfstr[dfstr['stroke'] == 1]
dfstrss
#Binh_Muc8
df.dropna(how='all', inplace=True)
df.head()

In [ ]:
len(dfstrss)


XÂY DỰNG MÔ HÌNH

In [ ]:
#Chau_muc2
#chia bộ dữ liệu thành tập huấn luyện và tập kiểm tra.
from sklearn.model_selection import train_test_split
#import lớp DecisionTreeClassifier từ thư viện scikit-learn. Đại diện cho mô hình cây quyết định.
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
# Chọn các cột làm đặc trưng và nhãn x: đặc trưng y là nhãn
X = df[['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status']]
y = df['stroke']
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
#loại bỏ cột stroke khỏi df, loại bỏ cột axis=1 nếu axis=0 thì loại bỏ hàng
x=df.drop(['stroke'],axis=1)
#tạo series y chứa các giá trị cột stroke trong df
y=df['stroke']
#tạo mô hình cây quyết định
model=DecisionTreeClassifier()
#huấn luyện mô hình
model.fit(X_train,y_train)
#Dự đoán trên tập kiểm tra
y_pred =model.predict(X_test)
#Đánh giá mô hình
print(classification_report(y_test,y_pred))
print("Độ chính xác của mô hình decision tree là: ", model.score(X_test, y_test)*100,"%")

In [ ]:
#Giang_Muc2

#import thu vien
import numpy as np
import seaborn as sns # mở cmd cài:  pip install seaborn
import matplotlib.pyplot as plt # mở cmd cài:  pip install matplotlib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
